In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
INPUT_BACKGROUND_FILES = '../input/stanford-background-dataset/images'
INPUT_FOREGROUND_FILES = '../input/augmented-cats-and-dogs'

OUTPUT_DATASET_ID = 'random-backgrounds-for-cats-and-dogs'
OUTPUT_DATASET_NAME = 'Random Backgrounds for Cats and Dogs'
OUTPUT_PATH = './output'

# kaggle_secrets not supported by Google Cloud Platform for Kaggle(Beta) at this time
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
#API_TOKEN = user_secrets.get_secret("Crop Cats and Cogs YOLOv3")

USER_ID = 'screamatthewind'
API_TOKEN = 'f6c53d46ab530b76a43aa3a3705a241f'

# same size is used in Augment Cats and Dogs
MIN_SCALE = 0.2
MAX_SCALE = 0.5

run_limit = 10

In [ ]:
import os
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [ ]:
import cv2
import numpy as np

# scale image by scale_factor, keep aspect ratio
def scale_image(image, scale_factor):
    
    image = np.array(image)
    image = cv2.resize(image, (0,0), fx=scale_factor, fy=scale_factor)

    image = Image.fromarray(image)
    
    return image

In [ ]:
import random
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

background_path, background_dir, background_files = next(os.walk(INPUT_BACKGROUND_FILES))
foreground_path, foreground_dir, foreground_files = next(os.walk(INPUT_FOREGROUND_FILES))

for i in range(run_limit):

    random_background_image_num = random.randrange(len(background_files))
    random_foreground_image_num = random.randrange(len(foreground_files))

    background_image_filename = background_path + '/' + background_files[random_background_image_num]
    foreground_image_filename = foreground_path + '/' + foreground_files[random_foreground_image_num]

    background_image = Image.open(background_image_filename)
    foreground_image = Image.open(foreground_image_filename)

    # composite randomly scaled forground image at random position on random background image
    scale_factor = random.uniform(MIN_SCALE, MAX_SCALE)
    foreground_image = scale_image(foreground_image, scale_factor)

    # paste foreground image onto background image at random position
    foreground_image_width, foreground_image_height = foreground_image.size
    background_image_width, background_image_height = background_image.size

    x_max_pos = background_image_width - foreground_image_height
    y_max_pos = background_image_height - foreground_image_height
    
    x_pos_factor = random.uniform(0, 1)
    y_pos_factor = random.uniform(0, 1)
    
    x_new_pos = int(x_max_pos * x_pos_factor)
    y_new_pos = int(y_max_pos * y_pos_factor)
    
    background_image = background_image.convert('RGBA') # make images have same number of color channels
    background_image.paste(foreground_image, (x_new_pos, y_new_pos), foreground_image) # 3rd parm is mask

    # draw bounding box 
    draw = ImageDraw.Draw(background_image)
    draw.rectangle(((x_new_pos, y_new_pos), (x_new_pos + foreground_image_width, y_new_pos + foreground_image_height)), outline=(255, 0, 0), width = 3)
    
    plt.imshow(background_image)   
    plt.show()
    
    # plt.imshow(foreground_image)   
    # plt.show()

    
    